In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import sys
import cv2

In [ ]:
# CONSTANTS
!wget https://raw.githubusercontent.com/jacobgil/keras-grad-cam/master/examples/cat_dog.png

In [2]:
def load_image(path):
    img = keras.preprocessing.image.load_img(path, target_size=(224, 224))
    x = keras.preprocessing.image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = keras.applications.vgg16.preprocess_input(x)
    return x

In [3]:
preprocessed_input = load_image("cat_dog.png")
model = keras.applications.vgg16.VGG16(weights='imagenet')
predictions = model.predict(preprocessed_input)    
indexx = np.argmax(predictions[0])
print(indexx)

242


In [4]:
top_1 = keras.applications.vgg16.decode_predictions(predictions)[0][0]
print('Predicted class:')
print('%s (%s) with probability %.2f' % (top_1[1], top_1[0], top_1[2]))


Predicted class:
boxer (n02108089) with probability 0.42


In [5]:
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
def target_category_loss(x, category_index, nb_classes):
    return tf.multiply(x, keras.backend.one_hot([category_index], nb_classes))

def target_category_loss_output_shape(input_shape):
    return input_shape

def normalize(x):
    return x / (keras.backend.sqrt(keras.backend.mean(keras.backend.square(x))) + 1e-5)

def compute_heatmap(input_model, image, layer_name, eps=1e-8):
    model = keras.models.Sequential()
    model.add(input_model)
    conv_output = [l for l in model.layers[0].layers if l.name is layer_name][0].output
    grad_model = keras.models.Model(
        inputs=[input_model.inputs],
        outputs=[conv_output, input_model.output]
    )
    
    with tf.GradientTape() as tape:
        inputs = tf.cast(image, tf.float32)
        (conv_outputs, predictions) = grad_model(inputs)
        loss = predictions[:, 242]
    
    grads = tape.gradient(loss, conv_outputs)
    
    
    cast_conv_outputs = tf.cast(conv_outputs > 0, tf.float32)
    cast_grads = tf.cast(grads > 0, tf.float32)
    
    guided_grads = cast_conv_outputs * cast_grads * grads

    conv_outputs = conv_outputs[0]
    guided_grads = guided_grads[0]
    
    weights = tf.reduce_mean(guided_grads, axis=(0, 1))
    cam = tf.reduce_sum(tf.multiply(weights, conv_outputs), axis=-1)
    
    heatmap = cv2.resize(cam.numpy(), (224, 224))
    
    numer = heatmap - np.min(heatmap)
    denom = (heatmap.max() - heatmap.min()) + 1e-5
    heatmap = numer / denom
    heatmap = (heatmap * 255).astype("uint8")
    return cam, heatmap

def overlay_heatmap(heatmap, image, alpha=0.5, color_map=cv2.COLORMAP_VIRIDIS):
    heatmap = cv2.applyColorMap(heatmap, color_map)
    output = cv2.addWeighted(image, alpha, heatmap, 1-alpha, 0)

    
    return output, heatmap

def grad_cam(input_model, image, category_index, layer_name, tape):
    
    model = keras.models.Sequential()
    model.add(input_model)

    nb_classes = 1000
    target_layer = lambda x: target_category_loss(
        x, category_index, nb_classes
    )
    model.add(keras.layers.Lambda(target_layer, output_shape=target_category_loss_output_shape))



    loss = keras.backend.sum(model.layers[-1].output)
    conv_output = [l for l in model.layers[0].layers if l.name is layer_name][0].output
    grads = tape.gradient(loss, conv_output)
    # print(grads)
    grads = normalize(grads[0])
    
    gradient_function = keras.backend.function([model.layers[0].input], [conv_output, grads])
    
    output, grads_val = gradient_function([image])
    output, grads_val = output[0, :], grads_val[0, :, :, :]
    
    weights = np.mean(grads_val, axis=(0, 1))
    cam = np.ones(output.shape[0:2], dtype=np.float32)
    
    for i, w in enumerate(weights):
        cam += w * output[:, :, i]
        
    cam = cv2.resize(cam, (224, 224))
    cam = np.maximum(cam, 0)
    heatmap = cam / np.max(cam)
    
    # Return to BGR from the preprocessed image
    image = image[0, :]
    image -= np.min(image)
    image = np.minimum(image, 255)
    
    cam = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    cam = np.float32(cam) + np.float32(image)
    cam = 255 * cam / np.max(cam)
    
    return np.uint8(cam), heatmap

In [ ]:
predicted_class = np.argmax(predictions)
#cam, heatmap = grad_cam(model, preprocessed_input, predicted_class, "block5_conv3", tape)
cam, heatmap = compute_heatmap(model, preprocessed_input, "block5_conv3")

heatmap = cv2.resize(heatmap, (224, 224))
(heatmap, output) = overlay_heatmap(heatmap, preprocessed_input, alpha=0.5)


In [ ]:
color_map = cv2.COLORMAP_VIRIDIS
heatmap = cv2.applyColorMap(heatmap, color_map)
output = cv2.addWeighted(load_image("boat.jpg"), 0.5, heatmap, 1 - 0.5, 0)
cv2.imwrite("gradcam.jpg", ouput)

In [ ]:
heatmap.shape

In [ ]:
preprocessed_input.shape

In [ ]:
np.squeeze(preprocessed_input).shape

In [ ]:
# https://keras.io/examples/vision/grad_cam/
# https://github.com/samson6460/tf-keras-gradcamplusplus/blob/master/gradcam.py
# https://github.com/eclique/keras-gradcam/blob/master/gradcam_vgg.ipynb

In [1]:
!wget https://upload.wikimedia.org/wikipedia/commons/e/ef/Brosen_motor_boat.jpg

--2020-06-30 19:13:14--  https://upload.wikimedia.org/wikipedia/commons/e/ef/Brosen_motor_boat.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 103.102.166.240
Connecting to upload.wikimedia.org (upload.wikimedia.org)|103.102.166.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 733416 (716K) [image/jpeg]
Saving to: ‘Brosen_motor_boat.jpg’

Brosen_motor_boat.j 100%[===================>] 716.23K  3.03MB/s    in 0.2s    

2020-06-30 19:13:15 (3.03 MB/s) - ‘Brosen_motor_boat.jpg’ saved [733416/733416]

